### read segmentation data 

In [1]:
import cellshape_helper as helper
from tifffile import imread
import napari
from skimage.measure import marching_cubes, regionprops
import trimesh
import numpy as np
import os

In [ ]:
membrane_dir = 'mari_second_data_5_frame/seg_timelapse_second_dataset.tif'
#nuclei_dir = 'data/test_nuclei.tif'
membrane_raw_dir = 'mari_second_data_5_frame/membrane_timelapse_second_dataset.tif'
#nuclei_raw_dir = 'data/nuclei_raw.tif'

membrane_seg = imread(membrane_dir)
#nuclei_seg = imread(nuclei_dir)
#membrane_raw = imread(membrane_raw_dir)
#nuclei_raw = imread(nuclei_raw_dir)

In [4]:
print(membrane_seg.shape)

(53, 17, 2439, 2439)


In [3]:
import torch

# Check if CUDA is available
cuda_available = torch.cuda.is_available()

# Get the name of the GPU
gpu_name = torch.cuda.get_device_name(0) if cuda_available else "No GPU available"

print("CUDA available:", cuda_available)
print("GPU Name:", gpu_name)

CUDA available: True
GPU Name: Quadro RTX 8000


### `tif_to_mesh` rewrite the function without saving

In [5]:
# marching_cubes require 3D input
vertices, faces, normals, values = marching_cubes(x)
mesh_obj = trimesh.Trimesh(
            vertices=vertices, faces=faces, process=False
        )
mesh_obj.export('data/test_nuclei_x.off')

In [10]:
from cellshape_helper.vendor.pytorch_geometric_files import read_off, sample_points
# dict pos, face
data = read_off('data/test_nuclei_x.off')
# change to .numpy() 
num_points = 1024
points = sample_points(data=data, num=num_points).numpy()

viewer = napari.Viewer()
viewer.add_image(nuclei_raw[0,6:14,1024:1040,895:910])
viewer.add_labels(x)

# Add the 3D mesh to the viewer
viewer.add_surface((vertices, faces, values), name="Marching Cubes Mesh")

# Add the sampled points to the viewer
viewer.add_points(points, size=1, name="Sampled Points")

### Creat single cell box

In [12]:
from skimage import measure
from tifffile import imwrite
from skimage import measure
import os
from tqdm import tqdm

def get_max_dims(seg_img):
    max_dims = np.array([0, 0, 0])
    for region in measure.regionprops(seg_img):
        minz, miny, minx, maxz, maxy, maxx = region.bbox
        dims = np.array([maxz-minz, maxy-miny, maxx-minx])
        max_dims = np.maximum(max_dims, dims)
    return max_dims

def get_max_dims_for_stack(image_stack):
    overall_max_dims = np.array([0, 0, 0])
    for frame in image_stack:
        max_dims = get_max_dims(frame)
        overall_max_dims = np.maximum(overall_max_dims, max_dims)
    return overall_max_dims

# Assuming membrane_seg is your image stack
overall_max_dims = get_max_dims_for_stack(membrane_seg)

# Print the maximum dimensions across all frames
print('The maximum size across all frames is:', overall_max_dims)

# Add a margin to the maximum dimensions
margin = np.array([5, 20, 20])  # Adjust the values as needed
max_dims_with_margin = overall_max_dims + margin
print('Maximum dimensions with margin:', max_dims_with_margin)


The maximum size across all frames is: [ 13 127 128]
Maximum dimensions with margin: [ 18 147 148]


In [19]:
def save_individual_label(seg_img, save_singlecell_dir, max_dims_with_margin,t):
    regions = measure.regionprops(seg_img)
    num_cells = len(regions)
    all_cells = np.zeros((num_cells, *max_dims_with_margin), dtype=seg_img.dtype)

def save_individual_label(seg, save_singlecell_dir, max_dims_with_margin):
    if not os.path.exists(save_singlecell_dir):
        os.makedirs(save_singlecell_dir)

    for t, seg_img in enumerate(seg):
        regions = measure.regionprops(seg_img)

        for i, region in tqdm(enumerate(regions), total=len(regions), desc=f"Processing Cells in Frame {t}"):
            minz, miny, minx, maxz, maxy, maxx = region.bbox
            cropped_image = seg_img[minz:maxz, miny:maxy, minx:maxx]
            label_mask = (cropped_image == region.label)

        padded_image = np.pad(label_mask, padding, mode='constant', constant_values=0)
        # Save the padded image with the label as the filename
        imwrite(f"{save_singlecell_dir}/{str(t)+region.label}.tif", padded_image)
        all_cells[i] = padded_image
            padding = [
                ((max_dims_with_margin[0] - label_mask.shape[0]) // 2,
                 max_dims_with_margin[0] - label_mask.shape[0] - ((max_dims_with_margin[0] - label_mask.shape[0]) // 2)),
                ((max_dims_with_margin[1] - label_mask.shape[1]) // 2,
                 max_dims_with_margin[1] - label_mask.shape[1] - ((max_dims_with_margin[1] - label_mask.shape[1]) // 2)),
                ((max_dims_with_margin[2] - label_mask.shape[2]) // 2,
                 max_dims_with_margin[2] - label_mask.shape[2] - ((max_dims_with_margin[2] - label_mask.shape[2]) // 2))
            ]

            padded_image = np.pad(label_mask, padding, mode='constant', constant_values=0)
            imwrite(f"{save_singlecell_dir}/{t}_{region.label}.tif", padded_image)



save_singlecell_dir = 'data/singlecell'
save_individual_label(membrane_seg, save_singlecell_dir, max_dims_with_margin)

Processing Cells in Frame 52: 100%|██████████| 3954/3954 [00:23<00:00, 167.37it/s]


### cellshape

#### read from crop pointcloud 

In [21]:
PATH_TO_TIF_FILES = "data/singlecell"
PATH_TO_SAVE_MESH = "data/mesh/"
PATH_TO_SAVE_PC = "data/pointcloud/"
NUM_POINTS = 1024

helper.tif_to_pc_directory(PATH_TO_TIF_FILES, 
                           PATH_TO_SAVE_MESH, 
                           PATH_TO_SAVE_PC, 
                           NUM_POINTS)

100%|██████████| 124226/124226 [1:54:36<00:00, 18.06it/s]  


In [2]:
import torch
from cellshape_cloud import CloudAutoEncoder

model = CloudAutoEncoder(num_features=128, 
                         k=20,
                         encoder_type="dgcnn",
                         decoder_type="foldingnet")



# model is not trained so far
points = torch.randn(1, 1024, 3)
recon, features = model(points)

#### To train an autoencoder on a set of point clouds created using cellshape-helper:

In [2]:
import torch
from torch.utils.data import DataLoader

import cellshape_cloud as cloud
from cellshape_cloud.vendor.chamfer_distance import ChamferLoss

input_dir = "data/pointcloud/"
batch_size = 64
learning_rate = 0.0001
num_epochs = 80
output_dir = "model_80epoch_5_frame/model_output/"

model = cloud.CloudAutoEncoder(num_features=128, 
                         k=20,
                         encoder_type="dgcnn",
                         decoder_type="foldingnet")


dataset = cloud.PointCloudDataset(input_dir)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [7]:
"""
according to cellshape_cloud/pointcloud_dataset.py class PointCloudDataset(Dataset)
each data point in your dataset is a tuple of four elements: (point_cloud, 0, 0, 0)

data_iter = iter(dataloader)
first_batch = next(data_iter)
first_batch_point_clouds = first_batch[0]

first_batch_point_clouds shape is (16,2048,3)
first_batch[1], first_batch[2], first_batch[4] is (16,) the value is 0


from cellshape_cloud.vendor import get_graph_feature
data_iter = iter(dataloader)
point_clouds = next(data_iter)
x = point_clouds[0]
x = x.transpose(2, 1)
batch_size = x.size(0)
graph_feature = get_graph_feature(x, k=40)
knn_graph = graph_feature.numpy()
knn_graph has shape (16,6,2048,20)
"""

In [3]:
# check cuda
# Assuming 'model' is your PyTorch model
if next(model.parameters()).is_cuda:
    print("Model is on GPU.")
else:
    print("Model is on CPU.")


Model is on CPU.


In [ ]:

criterion = ChamferLoss()

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate * 64 / batch_size,
    betas=(0.9, 0.999),
    weight_decay=1e-6,
)

#passing correct logging_infoto `clou_train` by
# Define paths and experiment name
name_logging = output_dir + "logging/directory"
name_model = output_dir + "save/model.pth"
name_writer = output_dir + "tensorboard/logs"
name = "my_experiment"

# Make sure directories exist
os.makedirs(name_logging, exist_ok=True)
os.makedirs(os.path.dirname(name_model), exist_ok=True)  # Since name_model includes the filename
os.makedirs(name_writer, exist_ok=True)

# Create the logging_info tuple
logging_info = (name_logging, name_model, name_writer, name)

cloud.train(model, dataloader, num_epochs, criterion, optimizer, logging_info)


Epoch 0: 100%|██████████| 1942/1942 [16:56<00:00,  1.91batch/s, loss=61.9]


Saving model to model_80epoch_5_frame/model_output/save/model.pth with loss = 159.59683896682535.
Finished epoch 0 with loss=159.59683896682535.


Epoch 1: 100%|██████████| 1942/1942 [15:50<00:00,  2.04batch/s, loss=41.8]


Saving model to model_80epoch_5_frame/model_output/save/model.pth with loss = 18.195672444776697.
Finished epoch 1 with loss=18.195672444776697.


Epoch 2: 100%|██████████| 1942/1942 [15:49<00:00,  2.04batch/s, loss=24]  


...

Epoch 38: 100%|██████████| 1942/1942 [17:03<00:00,  1.90batch/s, loss=12]  


Saving model to model_80epoch_5_frame/model_output/save/model.pth with loss = 8.161903343780157.
Finished epoch 38 with loss=8.161903343780157.


Epoch 39: 100%|██████████| 1942/1942 [15:50<00:00,  2.04batch/s, loss=13.7]


Saving model to model_80epoch_5_frame/model_output/save/model.pth with loss = 8.10238813690987.
Finished epoch 39 with loss=8.10238813690987.


Epoch 40:  73%|███████▎  | 1415/1942 [12:17<04:16,  2.05batch/s, loss=8.73]  